# Brain Cancer Classification Project
In this project, I will aim to design four models (Models: DenseNet121, EfficientNetV2S, EfficientNetV2M, InceptionResNetV2) that can correctly classify between a Glioma, a Meningioma and a Tumor from brain MRI scans.

### Import Libraries

In [15]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

tf.config.optimizer.set_experimental_options({'layout_optimizer': False})


AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [11]:
# Set seeds for reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

# Configuration
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)
EPOCHS = 15 
LEARNING_RATE = 1e-4 

DATA_DIR = '/kaggle/input/brain-cancer-mri-dataset/Brain_Cancer raw MRI data/Brain_Cancer'
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Quick check of directory structure to confirm path
print("\nDataset Contents:")
for root, dirs, files in os.walk(DATA_DIR):
    if len(files) > 0:
        print(f"{root} : {len(files)} files")
        break # Just print the first one found to verify

NameError: name 'tf' is not defined

### Data Augmentation & Preprocessing
Here we transform raw MRI images into normalised tensors for the model to comprehend. 

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Standardize size
    transforms.RandomHorizontalFlip(),   # Augmentation: flip images to increase variety
    transforms.RandomRotation(10),       # Augmentation: handle slight head tilts
    transforms.ToTensor(),               # Convert pixels to 0-1 scale
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet standards
])

# Validation transform is 'clean' for honest accuracy check
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
# Load all images using ImageFolder (detects 'Brain Tumor' and 'Healthy' folders)
train_set_full = datasets.ImageFolder(root=DATA_PATH, transform=train_transform)
val_set_full = datasets.ImageFolder(root=DATA_PATH, transform=val_transform)

FileNotFoundError: [Errno 2] No such file or directory: 'brain_cancer_mri'